# Interpolation

Interpolations are particularly useful when we repeatedly want to evaluate a function $f(x)$ that is computationally expensive to calculate. We then do the following

1. Calculate $f(x)$ values for a grid of $x$. This creates a "look-up" table.
2. In the expensive calculations, interpolate $f(x_i)$ by using the "look-up" table.

This notebook uses the package Interpolations (see https://github.com/JuliaMath/Interpolations.jl).

# Load Packages

In [1]:
using Interpolations

#using Dates             #Julia 0.7, needed for printmat.jl
include("printmat.jl")   #just a function for prettier matrix printing

printmatDate

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))    
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Some Data to Be Interpolated

As a simple illustration, we interpolate the sine function. (In practice, the interpolation technique is typically applied to more complicated functions.)

Notice that the approach below assumes that we supply $f(x)$ values for a *uniformly* spaced grid of $x$ values.

In [3]:
xGrid = linspace(-pi,pi,101)            #uniformly spaced grid
yGrid = sin.(xGrid)                     #y values at xGrid 

plot1 = plot(xGrid,yGrid,color=:red,linewidth=2,legend=nothing)
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 54.8916,195.781 60.1459,206.096 65.4001,216.371 70.6543,226.565 75.9086,236.637 81.1628,246.547 86.417,256.258 91.6713,265.73 96.9255,274.925 102.18,283.809 
 107.434,292.345 112.688,300.5 117.942,308.241 123.197,315.539 128.451,322.364 133.705,328.69 138.959,334.491 144.214,339.745 149.468,344.43 154.722,348.529 
 159.976,352.025 165.231,354.904 170.485,357.155 175.739,358.77 180.993,359.741 186.247,360.065 191.502,359.741 196.756,358.77 202.01,357.155 207.264,354.904 
 212.519,352.025 217.773,348.529 223.027,344.43 228.281,339.745 233.536,334.491 238.79,328.69 244.044,322.364 249.298,315.539 254.552,308.241 259.807,300.5 
 265.061,292.345 270.315,283.809 275.569,274.925 280.824,265.73 286.078,256.258 291.332,246.547 296.586,236.637 301.841,226.565 307.095,216.371 312.349,206.096 
 317.603,195.781 322.858,185.465 328.112,175.19 333.366,164.997 338.62,154.925 343.874,145.014 349.129,135.304 354.383,125.832 359.637,116.636 364.891,107.753 
 370.146,99.2166 375.4,91.0617 380.654,83.3201 385.908,76.0224 391.163,69.1972 396.417,62.8716 401.671,57.0706 406.925,51.817 412.179,47.1315 417.434,43.0327 
 422.688,39.5367 427.942,36.6574 433.196,34.406 438.451,32.7915 443.705,31.8202 448.959,31.4961 454.213,31.8202 459.468,32.7915 464.722,34.406 469.976,36.6574 
 475.23,39.5367 480.485,43.0327 485.739,47.1315 490.993,51.817 496.247,57.0706 501.501,62.8716 506.756,69.1972 512.01,76.0224 517.264,83.3201 522.518,91.0617 
 527.773,99.2166 533.027,107.753 538.281,116.636 543.535,125.832 548.79,135.304 554.044,145.014 559.298,154.925 564.552,164.997 569.806,175.19 575.061,185.465 
 580.315,195.781 
 "/>

## Preparing the Inter- and Extrapolation

The cell below applies some of the available options. 

The interpolation will based on a quadratic approximation. The ```scale()``` re-normalizes the $x$ values to "natural" units, that is, those that we used in ```xGrid``` to create the input ```yGrid```. (Motivation: ```itp``` normalizes the $x$ values to ```1:length(yGrid)```.)

The extrapolation values (outside the $x$ range in ```xGrid```) will be given the same values as end points of ```yGrid``` (flat extrapolation).

(Check out the documentation for more options.)

In [4]:
itp  = interpolate(yGrid, BSpline(Quadratic(Line())), OnGrid())
sitp = scale(itp,xGrid)                    #scaling to x has "natural" units, not 1:N

etp  = extrapolate(itp, Flat())            #flat outside observed range of x
setp = scale(etp,xGrid)

println()

## Interpolate

In [5]:
x  = [0.25;0.75]                     #to interpolate the y values at

#then interpolate as
y_interpolated = [sitp[x[i]] for i = 1:length(x)]      #most loop over x[i] elements

println("x, interpolated y values and true y values")
printmat([x y_interpolated sin.(x)])

x, interpolated y values and true y values
     0.250     0.247     0.247
     0.750     0.682     0.682



## Extrapolate

Extrapolation in- and outside xGrid (...but do we really want to do that?) is similar. Extrapolation inside the xGrid is similar to the interpolation (above).

In [6]:
x2             = [1.25;pi+0.1;pi+0.5]            #to extrapolate the y values at

y_extrapolated = [setp[x2[i]] for i = 1:length(x2)]

println("x2 and extrapolated values")
printmat([x2 y_extrapolated])

x2 and extrapolated values
     1.250     0.949
     3.242     0.000
     3.642     0.000



## Looking at the Results

In [7]:
plot2 = plot(xGrid,yGrid,color=:red,linewidth=2,label="sin function")
scatter!(x,y_interpolated,color=:magenta,markersize=5,marker=:square,label="interpolated")
scatter!(x2,y_extrapolated,color=:blue,markersize=8,label="extrapolated")
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 69.7621,195.781 74.3535,205.512 78.945,215.206 83.5364,224.822 88.1279,234.324 92.7193,243.674 97.3108,252.835 101.902,261.77 106.494,270.446 111.085,278.826 
 115.677,286.879 120.268,294.572 124.859,301.876 129.451,308.76 134.042,315.199 138.634,321.167 143.225,326.639 147.817,331.596 152.408,336.016 157,339.883 
 161.591,343.181 166.183,345.897 170.774,348.021 175.365,349.544 179.957,350.46 184.548,350.766 189.14,350.46 193.731,349.544 198.323,348.021 202.914,345.897 
 207.506,343.181 212.097,339.883 216.688,336.016 221.28,331.596 225.871,326.639 230.463,321.167 235.054,315.199 239.646,308.76 244.237,301.876 248.829,294.572 
 253.42,286.879 258.011,278.826 262.603,270.446 267.194,261.77 271.786,252.835 276.377,243.674 280.969,234.324 285.56,224.822 290.152,215.206 294.743,205.512 
 299.334,195.781 303.926,186.049 308.517,176.356 313.109,166.739 317.7,157.237 322.292,147.888 326.883,138.727 331.475,129.791 336.066,121.116 340.658,112.735 
 345.249,104.683 349.84,96.9892 354.432,89.6858 359.023,82.8011 363.615,76.3623 368.206,70.3948 372.798,64.9221 377.389,59.9659 381.981,55.5456 386.572,51.6788 
 391.163,48.3807 395.755,45.6644 400.346,43.5404 404.938,42.0173 409.529,41.101 414.121,40.7952 418.712,41.101 423.304,42.0173 427.895,43.5404 432.486,45.6644 
 437.078,48.3807 441.669,51.6788 446.261,55.5456 450.852,59.9659 455.444,64.9221 460.035,70.3948 464.627,76.3623 469.218,82.8011 473.81,89.6858 478.401,96.9892 
 482.992,104.683 487.584,112.735 492.175,121.116 496.767,129.791 501.358,138.727 505.95,147.888 510.541,157.237 515.133,166.739 519.724,176.356 524.315,186.049 
 528.907,195.781 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 sin function 
 
 
 
 
 interpolated 
 
 
 
 
 extrapolated